In [1]:
import pandas as pd
import numpy as np
import altair as alt
import locale

alt.data_transformers.enable('data_server')

#alt.data_transformers.enable('default')
#alt.data_transformers.disable_max_rows() # Warning: remove this after testing

locale.setlocale(locale.LC_ALL, 'sv_SE')
%config InlineBackend.figure_format = 'retina'

In [22]:
location="https://scb.se/hitta-statistik/statistik-efter-amne/befolkning/befolkningens-sammansattning/befolkningsstatistik/pong/tabell-och-diagram/preliminar-statistik-over-doda/"

source = pd.read_excel(location, sheet_name="Tabell 3", skiprows=5, header=1, usecols='A:W', na_values="..")
source['date'] = pd.to_datetime(source['Unnamed: 0'] + source['Year'].apply(str), format='%d %B%Y', errors='coerce')
source.drop(['Unnamed: 0', 'Year'], axis=1, inplace=True)
source.drop(source.index[[0,1]], inplace=True)
source.drop([850, 851, 852], inplace=True) # Drop summary rows

# This should really be done by combining column names with the first row. Brute force will do for now
rename_columns = {'Söderman-':'Södermanland', 'Öster-': 'Östergötland', 'Västra': 'Västra Götaland', 'Västman-': 'Västmanland', 'Västernorr-': 'Västernorrland', 'Väster-': 'Västerbotten', 'Norr-': 'Norrbotten'}
source.rename(columns=rename_columns, inplace=True)
source=source.set_index('date').stack().reset_index()
source.rename(columns={'level_1': 'county', 0: 'deaths'}, inplace=True)
source.deaths = source.deaths.astype(int)
source.county = source.county.astype('category')
weekly = source.groupby('county').resample('W', on='date').sum().reset_index()

In [10]:
df = weekly

selection = alt.selection_multi(fields=['county'], bind='legend')

alt.Chart(df, height=600, width=800).mark_bar().encode(
    alt.X('yearmonthdate(date):O'),
    alt.Y('sum(deaths):Q'),
    color='county:N',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(selection)

alt.Chart(...)

In [8]:
county_list = ['Skåne', 'Västra Götaland', 'Östergötland', 'Stockholm']

df = weekly
df = df[df['county'].isin(county_list)]

# Don't show last week due to lag in reporting
last_date_to_show = str(df.date.max() - pd.Timedelta('7d'))
current_filter = f'year(datum.date) == 2020 & datetime(datum.date) < datetime("{last_date_to_show}")'

base = alt.Chart(df, height=600, width=800, title="Weekly Deaths in Selected Counties 2020 compared to weekly average for 2018-2019").encode(
    color='county',
)

history = base.mark_point(opacity=0.4).encode(
    x='monthdate(date)',
    y='deaths',
).transform_filter(
    'year(datum.date) < 2020'
)

current = base.mark_line(point=True, interpolate='step-before').encode(
    x='monthdate(date)',
    y='deaths',
).transform_filter(
    current_filter
)

rule = base.mark_rule(opacity=0.5).encode(
    y=alt.Y('average(deaths)', title='Deaths'),
    size=alt.value(5)
).transform_filter(
    'year(datum.date) < 2020'
)

history + current + rule

alt.LayerChart(...)

In [23]:
df=source[source.date.notna()]

last_date_for_mean_calculation = str(df.date.max() - pd.Timedelta('7d'))
current = f'year(datum.date) == 2020 & datetime(datum.date) < datetime("{last_date_for_mean_calculation}")'
history_2019 = 'year(datum.date) == 2019'
history_2018 = 'year(datum.date) == 2018'

base = alt.Chart(
    df,
    height=600,
    width=800,
    title="Daily Deaths in Sweden 2018-2020"
).transform_aggregate(
    sum_deaths='sum(deaths)',
    groupby=['date']
).encode(
    x='monthdate(date)',
    color=alt.Color('year(date):N', title='Year', scale=alt.Scale(range=['#d9d9d9', '#b9b9b9', '#e6550d']))
)

points = base.mark_point().encode(
    y='sum_deaths:Q',
)

# https://vega.github.io/vega/docs/expressions/#datetime-functions
def plot_mean(time_filter):
    mean = base.transform_filter(
        time_filter
    ).transform_window(
        rolling_mean='mean(sum_deaths)',
        frame=[-10, 10]
    ).mark_line(
        size=4
    ).encode(
        y=alt.Y('rolling_mean:Q'),
    )

    return mean

plot_mean(history_2018) + plot_mean(history_2019) + points + plot_mean(current)

alt.LayerChart(...)

In [30]:
import altair as alt
from vega_datasets import data

source = data.seattle_weather()

scale = alt.Scale(domain=['sun', 'fog', 'drizzle', 'rain', 'snow'],
                  range=['#e7ba52', '#a7a7a7', '#aec7e8', '#1f77b4', '#9467bd'])
color = alt.Color('weather:N', scale=scale)

# We create two selections:
# - a brush that is active on the top panel
# - a multi-click that is active on the bottom panel
brush = alt.selection_interval(encodings=['x'])
click = alt.selection_multi(encodings=['color'])

# Top panel is scatter plot of temperature vs time
points = alt.Chart(source).mark_point().encode(
    alt.X('monthdate(date):T', title='Date'),
    alt.Y('temp_max:Q',
        title='Maximum Daily Temperature (C)',
        scale=alt.Scale(domain=[-5, 40])
    ),
    color=alt.condition(brush, color, alt.value('lightgray')),
    size=alt.Size('precipitation:Q', scale=alt.Scale(range=[5, 200]))
).properties(
    width=550,
    height=300
).add_selection(
    brush
).transform_filter(
    click
)

# Bottom panel is a bar chart of weather type
bars = alt.Chart(source).mark_bar().encode(
    x='count()',
    y='weather:N',
    color=alt.condition(click, color, alt.value('lightgray')),
).transform_filter(
    brush
).properties(
    width=550,
).add_selection(
    click
)

alt.vconcat(
    points,
    bars,
    data=source,
    title="Seattle Weather: 2012-2015"
)

alt.VConcatChart(...)

In [41]:
df = weekly

selection = alt.selection_multi(fields=['county'], bind='legend')

bars = alt.Chart(df, height=600, width=800).mark_bar().encode(
    alt.X('yearmonthdate(date):O'),
    alt.Y('sum(deaths):Q'),
    color='county:N',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(selection).transform_filter(selection)

bars

alt.Chart(...)

In [71]:
click = alt.selection_multi(encodings=['color'])

bars = alt.Chart(width=800).mark_bar().encode(
    alt.X('yearmonthdate(date):O'),
    alt.Y('sum(deaths):Q'),
    color='county:N',
    tooltip=['county', 'deaths']
).add_selection(click).transform_filter(click).interactive()

county_list = alt.Chart(width=800, height=300).mark_bar().encode(
    x='sum(deaths)',
    y='county:N',
    opacity=alt.condition(click, alt.value(1), alt.value(0.2)),
    color='county'
).add_selection(
    click
)

alt.vconcat(
    bars,
    county_list,
    data=df,
    title="Weekly Deaths in Sweden 2018-2020"
)

alt.VConcatChart(...)